# DONNEES DE PANEL

Application : prix du billet d'avion et concentration du marché du transport aérien.

Nous nous plaçons à la fin des années 1990. Il y a encore peu de compagnies d'aviation concurrentes à cette époque. Les autorités de la concurrence souhaitent savoir si le fait d'introduire de la concurrence permettrait de baisser les tarifs pratiqués par les compagnies aériennes. 

Nous disposons des données de 1149 trajets de vol interne aux États-Unis pour les quatre années suivantes : 1997, 1998, 1999, 2000 : 

$id$ : Identifiant du trajet de vol

$annee$ : Année

$tarif$ : Prix moyen du billet aller en dollars

$concen$ : Part de la compagnie la plus importante sur le marché (entre 0 et 1)

$dist$ : Distance du trajet de vol en km

Nous souhaitons savoir si la concentration d'un trajet particulier dans les mains d'une compagnie a une influence sur le prix qu'elle pratique. 

## 1. Structure de l'échantillon

Ouvrir les données **TD8_airfaremodified_v12.dta**.

Combien a-t-on d'observations au total ?

Quelle est l'unité individuelle, quelle est l'unité temporelle du panel ? 

A quoi ressemble donc la matrice des données de panel ? Utliser head().

Régresser le prix sur la concentration par MCO. Commenter. 

Produire un graphique mettant en relation la variable de concentration du marché et le prix du vol, en utilisant l'option **+ facet_wrap(~...)** pour différencier les 4 années, sur le sous échantillon des observations dont la distance de vol est inférieure ou égale à 2000 miles. 

Faire le même graphique sur le sous-échantillon des observations dont la distance de vol est supérieure à 2000 miles. 

Commenter. 

## 2. Analyse

### 2.1 MCO groupés (pooled OLS)
On souhaite estimer l'effet de la concentration sur le prix, en tenant compte de la distance du vol. 
Quelle est la variable dépendante ? La variable explicative principale ? Quel est le signe attendu du paramètre des différentes variables ? Ecrire l'équation à estimer. 

Estimer ce modèle par MCO. Toutes choses égales par ailleurs, quel est l'effet d'une distance de 100km supplémentaire ? D'une concentration de 10 points plus élevée ?

Un économétricien transformera toujours les données de prix et ici de distance en log. Donner les différentes raisons de ce choix. 

On ajoute donc au dataframe *ds* les données *ldist* et *ltarif* qui seront les données en logarithme de *dist* et *tarif*.

On estime ensuite le même modèle que précédemment, mais cette fois-ci avec les variables *ldist* et *lconcen*. Interpréter les coefficients.

Question subsidiaire : comment interpréter le coefficient d'un modèle niveau-log ?

On ajoute des effets fixes temporels. Rappeler la définition. Ecrire l'équation à estimer. 

<font color=green> 
Définition : $\eta_{t}$ = effets fixes temporels (autant de variables indicatrices binaires que de périodes, -1 pour éviter la colinéarité) qui captent la tendance temporelle générale de l'évolution de $tarif$ (évolution commune à tous les vols).

Equation : $log(tarif_{it}) = \gamma concen_{it} + \beta_{1} log(dist_{i})  + \eta_{t} + \epsilon_{it}$. 
</font>

On estime le modèle par MCO en ajoutant à notre fonction **lm()** précédente la variable "factor(*annee*)". 

a) Pourquoi pas la variable *annee* directement ? 

b) Estimer.

c) Interpréter les effets fixes temporels. 

Quels sont les deux problèmes posés par cette méthode d'estimation ? 

### 2.2 Modèles avec effets fixes individuels

Pour éviter ces deux problèmes potentiels, nous profitons de la structure de panel des données et nous estimons le modèle de régression avec effets fixes grâce à l'estimateur within. 

*Indice: on a besoin du package **plm** et de la fonction **plm(y~x, data=..., index=c("id","time", model="within")**.*

A quoi sert la commande index=c("id", "time") ? Que signifie ``model="within"'' ?

Quel est l'avantage de cette méthode d'estimation ?

Pourquoi ne peut-on pas utiliser la variable $dist$ ?

Interpréter le paramètre de $concen$.

On souhaite vérifier que l'estimateur within donne bien le même coefficient que l'**estimateur MCO qui inclut des effets fixes individuels et temporels**. Pour cela on estime par MCO le même modèle avec effets fixes temporels et individuels. 

Estimer et commenter.

De même, on souhaite vérifier que la transformation des données par soustraction de la moyenne individuelle donne bien le même coefficient. Grâce au code ci-dessous on crée la transformation de ltarif : 

In [ ]:
ds$mean_ltarif<-ave(ds$ltarif,ds$id)
ds$demeaned_ltarif<-ds$ltarif-ds$mean_ltarif

a) Faire de même pour concen. 

b) Estimer par MCO la régression de *demeaned_ltarif* sur *demeaned_lconcen* et les effets fixes temporels *factor(annee)*. 

## 3. Choix de la méthode d'estimation

On sait qu'il existe un autre type d'estimateur pour les données de panel : les **effets aléatoires**. 

L'**avantage des effets aléatoires sur les MCO groupés** est que l'autocorrélation intraindividuelle des erreurs est modélisée donc prise en compte. 

Quel est l'avantage des effets aléatoires sur les effets fixes (within) ? L'inconvénient ?

On estime le modèle avec effets aléatoires. Estimer et interpréter. 

*Indice: on utilise la même fonction **plm(y~x, data=..., index=c("id","time", model="...")**.*, mais avec le modèle **random** au lieu de **within**.

Le chercheur veut faire un choix entre effets fixes et aléatoires. Quel estimateur va-t-il choisir ?

*Indice : il compare les deux modèles avec un test de Hausman :* **phtest(model1,model2)** 